Implement the a priori algorithm. Apply to datasets indicated in code below.
You may work togehter, but you should make your own code. Share ideas about data structures and flow.

**A priori algorthm**

```
F_k: frequent k-itemsets

L_k: candidate k-itemsets

Algorithm
* Let k=1
* Generate F_1 = {frequent 1-itemsets}
* Repeat until F_k is empty
 * Candidate Generation: Generate L_(k+1) from F_k
 * Candidate Pruning: eliminate candidate itemsets in L_k+1 containing subsets of length k that are infrequent 
 * Support Counting: Count the support of each candidate in L_(k+1) by scanning the DB (or use a more efficient hash method)
 * Candidate Elimination: Eliminate candidates in L_(k+1) that are infrequent, leaving only those that are frequent => F_(k+1)
```





In [2]:

from google.colab import drive
drive.mount('/content/drive')
#!pip install mlxtend --upgrade


Mounted at /content/drive


In [3]:
#Functions and imports
#I had functions for candidate generation, candidate pruning, support count, and candidate elimination.
import pandas as pd
# from mlxtend.preprocessing import TransactionEncoder
# from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth
import numpy as np
import math
from itertools import combinations

def create_1item_candidate(trans):
  c1 = []
  for i in trans:
    for item in trans[i]:
      item = frozenset([item])
      if item not in c1:
        c1.append(item)


  return c1
    
def find_freq_item(trans, ck, min_sup):
  item_track = {}
  for i in trans:
    for item in ck: 
      if item.issubset(trans[i]):
        if item not in item_track:
          item_track[item] = 1
        else:
          item_track[item] += 1
  N_row = i
  freq_item = []
  item_sup = {}

  for item in item_track:
    supp = item_track[item] / N_row
    if supp >= min_sup:
      freq_item.append(item)

    item_sup[item] = supp

  return freq_item, item_sup


def candidate_k(freq_item, k):
  ck = []

  if k == 0:
    for f1, f2 in combinations(freq_item, 2):
      item = f1 | f2
      ck.append(item)
  else:
    for f1, f2 in combinations(freq_item, 2):
      intsec = f1 & f2
      if len(intsec) == k:
        item = f1 | f2
        if item not in ck: 
          ck.append(item)
  
  return ck 






In [4]:
#Main program. Some things below you may or may not want.


if __name__ == "__main__":

  #Transaction file name
  #trans_file_name = '/content/drive/MyDrive/CS Classes/CSC_373_Data_Mining/CSC_373_Student_Files/Data/GroceryStoreStacked.csv'
  #trans_file_name = '/content/drive/MyDrive/CS Classes/CSC_373_Data_Mining/CSC_373_Student_Files/Data/GroceryStoreStacked_sub.csv'
  #trans_file_name = '/content/drive/MyDrive/CS Classes/CSC_373_Data_Mining/CSC_373_Student_Files/Data/Table_5_1.txt'
  trans_file_name = '/content/drive/Shareddrives/CSC373_DMP_Wu_Chenyang/DMP_Association//Data/Copy of Assoc_Analysis_Vidhya.dat'
  print("Transaction file:", trans_file_name)
  #Set min_sup
  MIN_SUP = 0.1

  print("\nmin_sup",MIN_SUP)
  data_matrix = []
  trans = {}
  #data_matrix = pd.read_csv(trans_file_name)
  



  with open(trans_file_name,'r') as data_file_ptr:
    count = 0
    for dat_item in data_file_ptr:
      count +=1
      dat_item = dat_item.strip('\n')
      dat_item_list = dat_item.split(",")
      dat_item_list.pop(0)

      trans[count] = list(set(dat_item_list))
    #print(trans)
    c1 = create_1item_candidate(trans)
    freq_item, item_sup_all = find_freq_item(trans,c1,MIN_SUP)
    freq_items = [freq_item]
    cks = [c1]

    k = 0 
    while len(cks[k]) > 0 :
      freq_item = freq_items[k]
      ck = candidate_k(freq_item, k)
      freq_item, item_sup = find_freq_item(trans, ck, MIN_SUP)
      freq_items.append(freq_item)
      cks.append(ck)
      item_sup_all.update(item_sup)
      k += 1

    #print(freq_items)
    #print(cks)
    
    print("\n Number of transactions:", len(trans))
    print(item_sup_all)

    i = 1;
    for lists in cks:
      
      print("\n Candidate",i,"-itemset has ", len(lists), "items \n" )
      for supps in lists:
        print(" ", supps,": ",round(item_sup_all[supps],2),"  ", end = '')


      j = 0
      for supps in lists:
        if item_sup_all[supps] > MIN_SUP:
          j +=1
      

      for supps in lists:
        if item_sup_all[supps] > MIN_SUP:
          print(" ", supps,": ",round(item_sup_all[supps],2),"  ", end = '')
      

      print("\n \n Eliminated Candidate",i,"-itemset has ", len(freq_items[i-1]), "items \n" )

      for freq_supps in freq_items[i-1]:
        print(" ", freq_supps,": ",round(item_sup_all[freq_supps],2),"  ", end = '')
      print("")
      i +=1 
      
    

  



#print(data_matrix)
#print(frequent_itemsets)
  #My overall code flow followed the algorithm. I did build in some efficiencies
  #  for candidate generation.



Transaction file: /content/drive/Shareddrives/CSC373_DMP_Wu_Chenyang/DMP_Association//Data/Copy of Assoc_Analysis_Vidhya.dat

min_sup 0.1

 Number of transactions: 315
{frozenset({'Diaper'}): 0.40634920634920635, frozenset({'Cheese'}): 0.5015873015873016, frozenset({'Pencil'}): 0.3619047619047619, frozenset({'Meat'}): 0.47619047619047616, frozenset({'Bread'}): 0.5047619047619047, frozenset({'Eggs'}): 0.4380952380952381, frozenset({'Wine'}): 0.4380952380952381, frozenset({'Milk'}): 0.5015873015873016, frozenset({'Bagel'}): 0.4253968253968254, frozenset({''}): 0.09523809523809523, frozenset({'Diaper', 'Cheese'}): 0.2, frozenset({'Pencil', 'Diaper'}): 0.17142857142857143, frozenset({'Meat', 'Diaper'}): 0.19365079365079366, frozenset({'Bread', 'Diaper'}): 0.23174603174603176, frozenset({'Eggs', 'Diaper'}): 0.1619047619047619, frozenset({'Wine', 'Diaper'}): 0.23492063492063492, frozenset({'Pencil', 'Cheese'}): 0.2, frozenset({'Meat', 'Cheese'}): 0.3238095238095238, frozenset({'Bread', 'Chee

Some output - I think it is correct, but if you find problems, let me know

```
Transaction file: /content/drive/MyDrive/CS Classes/CSC_373_Data_Mining/CSC_373_Student_Files/Data/Table_5_1.txt

min_sup 2

Number of transactions: 5

Candidate 1-itemset has 6 items.
{'Bread': 4, 'Milk': 4, 'Beer': 3, 'Diapers': 4, 'Eggs': 1, 'Cola': 2}

Pruned 1-itemset is the same as the eliminated 1-itemset.

Eliminated 1-itemset has 5 items.
{'Bread': 4, 'Milk': 4, 'Beer': 3, 'Diapers': 4, 'Cola': 2}

Candidate 2-itemset has 10 items.
{'Bread,Milk': 0, 'Beer,Bread': 0, 'Bread,Diapers': 0, 'Bread,Cola': 0, 'Beer,Milk': 0, 'Diapers,Milk': 0, 'Cola,Milk': 0, 'Beer,Diapers': 0, 'Beer,Cola': 0, 'Cola,Diapers': 0}

Pruned 2-itemset has 10 items.
{'Bread,Milk': 0, 'Beer,Bread': 0, 'Bread,Diapers': 0, 'Bread,Cola': 0, 'Beer,Milk': 0, 'Diapers,Milk': 0, 'Cola,Milk': 0, 'Beer,Diapers': 0, 'Beer,Cola': 0, 'Cola,Diapers': 0}

Eliminated 2-itemset has 8 items.
{'Bread,Milk': 3, 'Beer,Bread': 2, 'Bread,Diapers': 3, 'Beer,Milk': 2, 'Diapers,Milk': 3, 'Cola,Milk': 2, 'Beer,Diapers': 3, 'Cola,Diapers': 2}

Candidate 3-itemset has 10 items.
{'Beer,Bread,Milk': 0, 'Bread,Diapers,Milk': 0, 'Bread,Cola,Milk': 0, 'Beer,Bread,Diapers': 0, 'Bread,Cola,Diapers': 0, 'Beer,Diapers,Milk': 0, 'Cola,Diapers,Milk': 0, 'Beer,Cola,Milk': 0, 'Beer,Cola,Diapers': 0, 'Beer,Bread,Cola': 0}

Pruned 3-itemset has 5 items.
{'Beer,Bread,Milk': 0, 'Bread,Diapers,Milk': 0, 'Beer,Bread,Diapers': 0, 'Beer,Diapers,Milk': 0, 'Cola,Diapers,Milk': 0}

Eliminated 3-itemset has 4 items.
{'Bread,Diapers,Milk': 2, 'Beer,Bread,Diapers': 2, 'Beer,Diapers,Milk': 2, 'Cola,Diapers,Milk': 2}

Candidate 4-itemset has 4 items.
{'Beer,Bread,Diapers,Milk': 0, 'Bread,Cola,Diapers,Milk': 0, 'Beer,Cola,Diapers,Milk': 0, 'Beer,Bread,Cola,Diapers': 0}

Pruned 4-itemset has 0 items.
{}

Eliminated 4-itemset has 0 items.
{}
END

```